## MLib - Machine learning library

In [ ]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#from pyspark.ml.feature import Tokenizer
#from pyspark.ml.feature import StopWordsRemover
#from pyspark.ml.feature import HashingTF
#from pyspark.ml.feature import IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel
from pyspark.sql.types import *

In [ ]:
# input and output folders
trainingData = "trainingData.csv"
unlabeledData = "unlabeledData.csv"
outputPath = "./"

In [ ]:
# *************************
# Training step
# *************************

# Create a DataFrame from trainingData.csv
# Training data in raw format
trainingData = spark.read.load(trainingData,\
                     format="csv",\
                     header=True,\
                     inferSchema=True)

In [ ]:
trainingData.printSchema()
trainingData.show()

In [ ]:
root
 |-- label: integer (nullable = true)
 |-- text: string (nullable = true)

+-----+--------------------+
|label|                text|
+-----+--------------------+
|    1|The Spark system ...|
|    1|Spark is a new di...|
|    0|Turin is a beauti...|
|    0|Turin is in the n...|
+-----+--------------------+

In [ ]:
# Define a Python function that returns the number of words occurring in the input string
def countWords(text):
    return len(text.split(" "))

In [ ]:
# Register a UDF function associated with countWords
# We explicitly report also the data type of the returned value
spark.udf.register("countWords", countWords, IntegerType())

In [ ]:
# Define a Python function that checks if the input string contain the work "Spark"
def containsSpark(text):
    return text.find("Spark")>=0

In [ ]:
# Register a UDF function associated with containsSpark
# We explicitly report also the data type of the returned value
spark.udf.register("containsSpark", containsSpark, BooleanType())

In [ ]:
# Select the attributes label and text and create  two new columns:
# numLines and SparkWord
newFeaturesDF = trainingData\
.selectExpr("label", "text", "countWords(text)", "containsSpark(text)")\
.withColumnRenamed("countWords(text)", "numLines")\
.withColumnRenamed("containsSpark(text)", "SparkWord")

# OR
# Create an SQLTransformer to add two column to the input dataframe:
# numLines and SparkWord
sqlTrans = SQLTransformer(statement="""SELECT *,
countWords(text) AS numLines,
containsSpark(text) AS SparkWord
FROM __THIS__""")

In [ ]:
newFeaturesDF.printSchema()
newFeaturesDF.show()

In [ ]:
root
 |-- label: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- numLines: integer (nullable = true)
 |-- SparkWord: boolean (nullable = true)

+-----+--------------------+--------+---------+
|label|                text|numLines|SparkWord|
+-----+--------------------+--------+---------+
|    1|The Spark system ...|       7|     true|
|    1|Spark is a new di...|       6|     true|
|    0|Turin is a beauti...|       5|    false|
|    0|Turin is in the n...|       8|    false|
+-----+--------------------+--------+---------+

In [ ]:
# Use an assembler to combine "numLines" and "SparkWord" in a Vector
assembler = VectorAssembler(inputCols=["numLines", "SparkWord"],\
                            outputCol="features")

In [ ]:
# Create a classification model based on the logistic regression algorithm
# We can set the values of the parameters of the 
# Logistic Regression algorithm using the setter methods.
lr = LogisticRegression()\
.setMaxIter(10)\
.setRegParam(0.01)

In [ ]:
# Define the pipeline that is used to create the logistic regression
# model on the training data.
pipeline = Pipeline().setStages([assembler, lr])

In [ ]:
# Execute the pipeline on the training data to build the 
# classification model
classificationModel = pipeline.fit(newFeaturesDF)

# Now, the classification model can be used to predict the class label
# of new unlabeled data

In [ ]:
# *************************
# Prediction  step
# *************************
# Read unlabeled data
# Create a DataFrame from unlabeledData.csv
# Unlabeled data in raw format
unlabeledData = spark.read.load(unlabeledData,\
                     format="csv", header=True, inferSchema=True)

In [ ]:
#unlabeledData.printSchema()
#unlabeledData.show()

In [ ]:
newFeaturesDFunlabeled = unlabeledData\
.selectExpr("label", "text", "countWords(text)", "containsSpark(text)")\
.withColumnRenamed("countWords(text)", "numLines")\
.withColumnRenamed("containsSpark(text)", "SparkWord")

In [ ]:
newFeaturesDFunlabeled.printSchema()
newFeaturesDFunlabeled.show()

In [ ]:
root
 |-- label: string (nullable = true)
 |-- text: string (nullable = true)
 |-- numLines: integer (nullable = true)
 |-- SparkWord: boolean (nullable = true)

+-----+--------------------+--------+---------+
|label|                text|numLines|SparkWord|
+-----+--------------------+--------+---------+
| null|Spark performs be...|       5|     true|
| null|Comparison betwee...|       5|     true|
| null|Turin is in Piedmont|       4|    false|
+-----+--------------------+--------+---------+

In [ ]:
# Make predictions on unlabeled documents by using the 
# Transformer.transform() method.
# The transform will only use the 'features' columns
predictionsDF = classificationModel.transform(newFeaturesDFunlabeled)

In [ ]:
predictionsDF.printSchema()
predictionsDF.show()

In [ ]:
root
 |-- label: string (nullable = true)
 |-- text: string (nullable = true)
 |-- numLines: integer (nullable = true)
 |-- SparkWord: boolean (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

+-----+--------------------+--------+---------+---------+--------------------+--------------------+----------+
|label|                text|numLines|SparkWord| features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------+---------+---------+--------------------+--------------------+----------+
| null|Spark performs be...|       5|     true|[5.0,1.0]|[-3.1272480248757...|[0.04199718899423...|       1.0|
| null|Comparison betwee...|       5|     true|[5.0,1.0]|[-3.1272480248757...|[0.04199718899423...|       1.0|
| null|Turin is in Piedmont|       4|    false|[4.0,0.0]|[3.19966999960023...|[0.96082185681571...|       0.0|
+-----+--------------------+--------+---------+---------+--------------------+--------------------+----------+

In [ ]:
# The returned DataFrame has the following schema (attributes)
# |-- label: string (nullable = true)
# |-- text: string (nullable = true)
# |-- words: array (nullable = true)
# |    |-- element: string (containsNull = true)
# |-- filteredWords: array (nullable = true)
# |    |-- element: string (containsNull = true)
# |-- rawFeatures: vector (nullable = true)
# |-- features: vector (nullable = true)
# |-- rawPrediction: vector (nullable = true)
# |-- probability: vector (nullable = true)
# |-- prediction: double (nullable = false)

# Select only the original features (i.e., the value of the original text attribute) and 
# the predicted class for each record
predictions = predictionsDF.select("text", "prediction")

In [ ]:
predictions.printSchema()
predictions.show(truncate=False)

In [ ]:
root
 |-- text: string (nullable = true)
 |-- prediction: double (nullable = false)

+-----------------------------------+----------+
|text                               |prediction|
+-----------------------------------+----------+
|Spark performs better than Hadoop  |1.0       |
|Comparison between Spark and Hadoop|1.0       |
|Turin is in Piedmont               |0.0       |
+-----------------------------------+----------+

In [ ]:
# Save the result in an HDFS output folder
predictions.write.csv(outputPath, header="true")

## GraphFrame

### Compute the number of followers

In [ ]:
from graphframes import GraphFrame

In [ ]:
inputPathVertexes = "vertexes.csv"
inputPathEdges = "edges.csv"
outputPath = "./"

In [ ]:
# Read the content of vertexes.csv
vDF = spark.read.load(inputPathVertexes,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
vDF.printSchema()
vDF.show()

In [ ]:
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)

+---+-----+---+
| id| name|age|
+---+-----+---+
| u1|Alice| 34|
| u2|  Bob| 36|
| u3| John| 30|
| u4|David| 29|
| u5| Paul| 32|
| u6| Adel| 36|
| u7| Eddy| 60|
+---+-----+---+

In [ ]:
# Read the content of edges.csv
eDF = spark.read.load(inputPathEdges,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
eDF.printSchema()
eDF.show()

In [ ]:
root
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- linktype: string (nullable = true)

+---+---+--------+
|src|dst|linktype|
+---+---+--------+
| u1| u2|  friend|
| u1| u4|  friend|
| u1| u5|  friend|
| u2| u1|  friend|
| u2| u3|  follow|
| u3| u2|  follow|
| u4| u1|  friend|
| u4| u5|  friend|
| u5| u1|  friend|
| u5| u4|  friend|
| u5| u6|  follow|
| u6| u3|  follow|
| u7| u6|  follow|
+---+---+--------+

In [ ]:
# Only the "follow" edges are useful
# Filter the input edge dataframe before creating the graph
filteredEdges = eDF.filter("linktype='follow' ")

In [ ]:
# Create the input graph
g = GraphFrame(vDF, filteredEdges)

In [ ]:
# Count the number of followers for each user (i.e., the number of in-links) 
userNumFollowersDF = g.inDegrees.withColumnRenamed("inDegree","numFollowers")

In [ ]:
userNumFollowersDF.printSchema()
userNumFollowersDF.show()

In [ ]:
root
 |-- id: string (nullable = true)
 |-- numFollowers: integer (nullable = false)

+---+------------+
| id|numFollowers|
+---+------------+
| u3|           2|
| u6|           2|
| u2|           1|
+---+------------+

In [ ]:
# Save the result in the output folder
userNumFollowersDF.write.csv(outputPath, header=True)

### Extract users with the maximum number of followers

In [ ]:
inputPathVertexes = "vertexes.csv"
inputPathEdges = "edges.csv"
outputPath = "./"

In [ ]:
# Read the content of vertexes.csv
vDF = spark.read.load(inputPathVertexes,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
# Read the content of edges.csv
eDF = spark.read.load(inputPathEdges,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
# Only the "follow" edges are useful
# Filter the input edge dataframe before creating the graph
filteredEdfes = eDF.filter("linktype='follow' ")

In [ ]:
# Create the input graph
g = GraphFrame(vDF, filteredEdfes)

In [ ]:
# Count the number of followers for each user (i.e., the number of in-links) 
userNumFollowersDF = g.inDegrees.withColumnRenamed("inDegree","numFollowers")

In [ ]:
# Select the user(s) with the maximum number of followers

In [ ]:
# Compute the maximum value of "number of followers"
maxFollowersDF = userNumFollowersDF.agg({"numFollowers":"max"})\
.withColumnRenamed("max(numFollowers)","maxFollowers")

In [ ]:
# Select the single record/Row of the DataFrame maxFollowersDF
rowMaxNumFollowers = maxFollowersDF.first()

In [ ]:
# Retrieve the maximum number of followers from rowMaxNumFollowers
maxNumFollowers=rowMaxNumFollowers.maxFollowers

In [ ]:
# Select the user(s) with the maximum number of followers
selectedUsersDF=userNumFollowersDF.filter(userNumFollowersDF.numFollowers==maxNumFollowers)
# Or
#electedUsersDF=userNumFollowersDF.filter("numFollowers="+str(maxNumFollowers))

In [ ]:
selectedUsersDF.printSchema()
selectedUsersDF.show()

In [ ]:
root
 |-- id: string (nullable = true)
 |-- numFollowers: integer (nullable = false)

+---+------------+
| id|numFollowers|
+---+------------+
| u3|           2|
| u6|           2|
+---+------------+

In [ ]:
# Save the result in the output folder
selectedUsersDF.write.csv(outputPath, header=True)

## Motif finding

In [ ]:
inputPathVertexes = "vertexes.csv"
inputPathEdges = "edges.csv"
outputPath = "./"

In [ ]:
# Read the content of vertexes.csv
vDF = spark.read.load(inputPathVertexes,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
# Read the content of edges.csv
eDF = spark.read.load(inputPathEdges,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
# Select only links of type friend
filteredEdges = eDF.filter("linktype='friend' ")

In [ ]:
# Create the input graph
g = GraphFrame(vDF, filteredEdges)

In [ ]:
# Select the paths/motifs (v1)-[]->(v2);!(v2)-[]->(v1)
selectedPaths = g.find("(userx)-[]->(usery);!(usery)-[]->(userx)")

In [ ]:
selectedPaths.printSchema()
selectedPaths.show()

In [ ]:
root
 |-- userx: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- age: integer (nullable = true)
 |-- usery: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- age: integer (nullable = true)

+---------------+---------------+
|          userx|          usery|
+---------------+---------------+
|[u4, David, 29]|[u1, Alice, 34]|
|[u1, Alice, 34]|  [u2, Bob, 36]|
+---------------+---------------+

In [ ]:
# Select only the ids of the users (rename the selected columns)
selectedPairsDF=selectedPaths.selectExpr("userx.id as IdFriend","usery.id as IdNotFriend")

In [ ]:
selectedPairsDF.printSchema()
selectedPairsDF.show()

In [ ]:
root
 |-- IdFriend: string (nullable = true)
 |-- IdNotFriend: string (nullable = true)

+--------+-----------+
|IdFriend|IdNotFriend|
+--------+-----------+
|      u4|         u1|
|      u1|         u2|
+--------+-----------+

In [ ]:
# Save the result in the output folder
selectedPairsDF.write.csv(outputPath, header=True)

### Select users and topics of interest

In [ ]:
inputPathVertexes = "vertexes.csv"
inputPathEdges = "edges.csv"
outputPath = "./"

In [ ]:
# Read the content of vertexes.csv
vDF = spark.read.load(inputPathVertexes,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
vDF.printSchema()
vDF.show()

In [ ]:
root
 |-- id: string (nullable = true)
 |-- entityName: string (nullable = true)
 |-- name: string (nullable = true)

+---+----------+--------+
| id|entityName|    name|
+---+----------+--------+
| V1|      user|   Paolo|
| V2|     topic|     SQL|
| V3|      user|   David|
| V4|     topic|Big Data|
| V5|      user|    John|
+---+----------+--------+

In [ ]:
# Read the content of edges.csv
eDF = spark.read.load(inputPathEdges,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
eDF.printSchema()
eDF.show()

In [ ]:
root
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- linktype: string (nullable = true)

+---+---+----------+
|src|dst|  linktype|
+---+---+----------+
| V1| V2|      like|
| V1| V3|    follow|
| V1| V4|    follow|
| V3| V2|      like|
| V3| V4|      like|
| V5| V2|  expertOf|
| V2| V4|correlated|
| V4| V2|correlated|
+---+---+----------+

In [ ]:
# Only the "follow" edges are useful
# Filter the input edge dataframe before creating the graph
filteredEdfes = eDF.filter("linktype='follow' ")

In [ ]:
# Create the input graph
g = GraphFrame(vDF, filteredEdfes)

In [ ]:
pathsDF = g.find("(v1)-[]->(v2)")

In [ ]:
pathsDF.printSchema()
pathsDF.show()

In [ ]:
root
 |-- v1: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- entityName: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- v2: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- entityName: string (nullable = true)
 |    |-- name: string (nullable = true)

+-----------------+--------------------+
|               v1|                  v2|
+-----------------+--------------------+
|[V1, user, Paolo]|   [V3, user, David]|
|[V1, user, Paolo]|[V4, topic, Big D...|
|[V3, user, David]|    [V2, topic, SQL]|
|[V3, user, David]|[V4, topic, Big D...|
+-----------------+--------------------+

In [ ]:
# Select the pair (user,topic)
selectedPathsDF = pathsDF.filter("v1.entityName='user' AND v2.entityName='topic' ")

In [ ]:
selectedPathsDF.printSchema()
selectedPathsDF.show()

In [ ]:
root
 |-- v1: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- entityName: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- v2: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- entityName: string (nullable = true)
 |    |-- name: string (nullable = true)

+-----------------+--------------------+
|               v1|                  v2|
+-----------------+--------------------+
|[V1, user, Paolo]|[V4, topic, Big D...|
|[V3, user, David]|    [V2, topic, SQL]|
|[V3, user, David]|[V4, topic, Big D...|
+-----------------+--------------------+

In [ ]:
# Select name of the user and "name" of the topic
userTopicDF = selectedPathsDF.selectExpr("v1.name as username", "v2.name as topic")

In [ ]:
userTopicDF.show()

In [ ]:
+--------+--------+
|username|   topic|
+--------+--------+
|   Paolo|Big Data|
|   David|     SQL|
|   David|Big Data|
+--------+--------+

In [ ]:
# Save the result in the output folder
userTopicDF.write.csv(outputPath, header=True)

### Filter users by edge type and topic

In [ ]:
inputPathVertexes = "vertexes.csv"
inputPathEdges = "edges.csv"
outputPath = "./"

In [ ]:
# Read the content of vertexes.csv
vDF = spark.read.load(inputPathVertexes,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
vDF.printSchema()
vDF.show()

In [ ]:
root
 |-- id: string (nullable = true)
 |-- entityName: string (nullable = true)
 |-- name: string (nullable = true)

+---+----------+--------+
| id|entityName|    name|
+---+----------+--------+
| V1|      user|   Paolo|
| V2|     topic|     SQL|
| V3|      user|   David|
| V4|     topic|Big Data|
| V5|      user|    John|
+---+----------+--------+

In [ ]:
# Read the content of edges.csv
eDF = spark.read.load(inputPathEdges,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
eDF.printSchema()
eDF.show()

In [ ]:
root
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- linktype: string (nullable = true)

+---+---+----------+
|src|dst|  linktype|
+---+---+----------+
| V1| V2|      like|
| V1| V3|    follow|
| V1| V4|    follow|
| V3| V2|    follow|
| V3| V4|    follow|
| V5| V2|  expertOf|
| V2| V4|correlated|
| V4| V2|correlated|
+---+---+----------+

In [ ]:
# Only the "follow"  and "correlated" edges are useful
# Filter the input edge dataframe before creating the graph
filteredEdfes = eDF.filter("linktype='follow' OR linktype='correlated' ")

In [ ]:
# Create the input graph
g = GraphFrame(vDF, filteredEdfes)

In [ ]:
pathsDF = g.find("(v1)-[e1]->(v2);(v2)-[e2]->(v3)")

In [ ]:
pathsDF.printSchema()
pathsDF.show()

In [ ]:
root
 |-- v1: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- entityName: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- e1: struct (nullable = false)
 |    |-- src: string (nullable = true)
 |    |-- dst: string (nullable = true)
 |    |-- linktype: string (nullable = true)
 |-- v2: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- entityName: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- e2: struct (nullable = false)
 |    |-- src: string (nullable = true)
 |    |-- dst: string (nullable = true)
 |    |-- linktype: string (nullable = true)
 |-- v3: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- entityName: string (nullable = true)
 |    |-- name: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  v1|                  e1|                  v2|                  e2|                  v3|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|   [V1, user, Paolo]|    [V1, V3, follow]|   [V3, user, David]|    [V3, V4, follow]|[V4, topic, Big D...|
|   [V1, user, Paolo]|    [V1, V3, follow]|   [V3, user, David]|    [V3, V2, follow]|    [V2, topic, SQL]|
|   [V1, user, Paolo]|    [V1, V4, follow]|[V4, topic, Big D...|[V4, V2, correlated]|    [V2, topic, SQL]|
|   [V3, user, David]|    [V3, V2, follow]|    [V2, topic, SQL]|[V2, V4, correlated]|[V4, topic, Big D...|
|   [V3, user, David]|    [V3, V4, follow]|[V4, topic, Big D...|[V4, V2, correlated]|    [V2, topic, SQL]|
|    [V2, topic, SQL]|[V2, V4, correlated]|[V4, topic, Big D...|[V4, V2, correlated]|    [V2, topic, SQL]|
|[V4, topic, Big D...|[V4, V2, correlated]|    [V2, topic, SQL]|[V2, V4, correlated]|[V4, topic, Big D...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

In [ ]:
# Select the triples user -> follow -> topic -> correlated -> topic="Big data"
selectedPathsDF = pathsDF.filter("""v1.entityName='user' 
AND e1.linktype='follow'
AND v2.entityName='topic'
AND e2.linktype='correlated'
AND v3.entityName='topic' AND v3.name='Big Data' """)

In [ ]:
selectedPathsDF.printSchema()
selectedPathsDF.show()

In [ ]:
root
 |-- v1: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- entityName: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- e1: struct (nullable = false)
 |    |-- src: string (nullable = true)
 |    |-- dst: string (nullable = true)
 |    |-- linktype: string (nullable = true)
 |-- v2: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- entityName: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- e2: struct (nullable = false)
 |    |-- src: string (nullable = true)
 |    |-- dst: string (nullable = true)
 |    |-- linktype: string (nullable = true)
 |-- v3: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- entityName: string (nullable = true)
 |    |-- name: string (nullable = true)

+-----------------+----------------+----------------+--------------------+--------------------+
|               v1|              e1|              v2|                  e2|                  v3|
+-----------------+----------------+----------------+--------------------+--------------------+
|[V3, user, David]|[V3, V2, follow]|[V2, topic, SQL]|[V2, V4, correlated]|[V4, topic, Big D...|
+-----------------+----------------+----------------+--------------------+--------------------+

In [ ]:
# Select name of the selected users
usersDF = selectedPathsDF.selectExpr("v1.name as username")

In [ ]:
usersDF.show()

In [ ]:
+--------+
|username|
+--------+
|   David|
+--------+

In [ ]:
# Save the result in the output folder
usersDF.write.csv(outputPath, header=True)

### Select closest nodes to user1

In [ ]:
inputPathVertexes = "vertexes.csv"
inputPathEdges = "edges.csv"
outputPath = "./"

In [ ]:
# Read the content of vertexes.csv
vDF = spark.read.load(inputPathVertexes,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
vDF.printSchema()
vDF.show()

In [ ]:
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)

+---+-----+---+
| id| name|age|
+---+-----+---+
| u1|Alice| 34|
| u2|  Bob| 36|
| u3| John| 30|
| u4|David| 29|
| u5| Paul| 32|
| u6| Adel| 36|
| u7| Eddy| 60|
+---+-----+---+

In [ ]:
# Read the content of edges.csv
eDF = spark.read.load(inputPathEdges,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
eDF.printSchema()
eDF.show()

In [ ]:
root
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- linktype: string (nullable = true)

+---+---+--------+
|src|dst|linktype|
+---+---+--------+
| u1| u2|  friend|
| u1| u4|  friend|
| u1| u5|  friend|
| u2| u1|  friend|
| u2| u3|  follow|
| u3| u2|  follow|
| u4| u1|  friend|
| u4| u5|  friend|
| u5| u1|  friend|
| u5| u4|  friend|
| u5| u6|  follow|
| u6| u3|  follow|
+---+---+--------+

In [ ]:
# Create the input graph
g = GraphFrame(vDF, eDF)

In [ ]:
# Compute for each vertex the length of the shortest path to u1
shortPathsLengDF = g.shortestPaths(['u1'])

In [ ]:
shortPathsLengDF.printSchema()
shortPathsLengDF.show()

In [ ]:
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- distances: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = false)

+---+-----+---+---------+
| id| name|age|distances|
+---+-----+---+---------+
| u6| Adel| 36|[u1 -> 3]|
| u3| John| 30|[u1 -> 2]|
| u2|  Bob| 36|[u1 -> 1]|
| u4|David| 29|[u1 -> 1]|
| u5| Paul| 32|[u1 -> 1]|
| u1|Alice| 34|[u1 -> 0]|
| u7| Eddy| 60|       []|
+---+-----+---+---------+

In [ ]:
# Select only the users who can reach u1 in less than 3 "hops"
selectedUsersDF=shortPathsLengDF.filter("distances['u1']<3 AND id<>'u1' ")

In [ ]:
selectedUsersDF.printSchema()
selectedUsersDF.show()

In [ ]:
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- distances: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = false)

+---+-----+---+---------+
| id| name|age|distances|
+---+-----+---+---------+
| u3| John| 30|[u1 -> 2]|
| u2|  Bob| 36|[u1 -> 1]|
| u4|David| 29|[u1 -> 1]|
| u5| Paul| 32|[u1 -> 1]|
+---+-----+---+---------+

In [ ]:
# Create a DataFrame with Columns name and numHops
usersNameNumHopsDF=selectedUsersDF.selectExpr("name", "distances['u1'] AS numHops")

In [ ]:
usersNameNumHopsDF.printSchema()
usersNameNumHopsDF.show()

In [ ]:
root
 |-- name: string (nullable = true)
 |-- numHops: integer (nullable = true)

+-----+-------+
| name|numHops|
+-----+-------+
| John|      2|
|  Bob|      1|
|David|      1|
| Paul|      1|
+-----+-------+

In [ ]:
# Save the result in the output folder
usersNameNumHopsDF.write.csv(outputPath, header=True)